# 📦 BRONZE LAYER - Extract depuis sources DIRECTES

**Flux correct ETL** : Extract → Transform → Load

**Auteurs** : Nejma MOUALHI | Brieuc OLIVIERI | Nicolas TAING

---

## 🎯 Sources de données

### 1. PostgreSQL (tables originales) :
- Patient, Consultation, Diagnostic, Professionnel_de_sante, etc.
- **AAAA + date** : Données d'hospitalisation (82K lignes)

### 2. CSV BRUTS (lus directement depuis /DATA_2024/) :
- ✅ Établissements de santé (416K lignes)
- ✅ Satisfaction 2019 (1K lignes)
- ✅ **Décès 2019 UNIQUEMENT** (600K lignes - FILTRÉ depuis 25M)

**Important** : Les CSV sont lus **DIRECTEMENT** depuis le système de fichiers, **PAS** depuis PostgreSQL !

**Nouveau** : Le fichier `deces.csv` est maintenant **FILTRÉ** pour ne garder que **2019** (gain de 98% de performance).

In [1]:
# Imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, lit, year, to_date, col
from datetime import datetime
import time

print("✅ Imports OK")

✅ Imports OK


In [2]:
# Configuration Spark avec plus de mémoire pour les gros fichiers
spark = SparkSession.builder \
    .appName("CHU_Bronze_Extract_Sources_Directes") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

print(f"✅ Spark {spark.version} démarré")
print(f"📊 Master: {spark.sparkContext.master}")

✅ Spark 3.5.0 démarré
📊 Master: local[*]


In [3]:
# Configuration PostgreSQL pour les tables originales
JDBC_URL = "jdbc:postgresql://chu_postgres:5432/healthcare_data"
JDBC_PROPS = {
    "user": "admin",
    "password": "admin123",
    "driver": "org.postgresql.Driver"
}

# Tables PostgreSQL originales (13 tables)
POSTGRES_TABLES = [
    "Patient",
    "Consultation",
    "Diagnostic",
    "Professionnel_de_sante",
    "Mutuelle",
    "Adher",
    "Prescription",
    "Medicaments",
    "Laboratoire",
    "Salle",
    "Specialites",
    "date",
    "AAAA"
]

# Configuration chemins
DATA_DIR = "/home/jovyan/DATA_2024"
OUTPUT_BASE = "/home/jovyan/data/bronze"

print(f"✅ {len(POSTGRES_TABLES)} tables PostgreSQL à extraire")
print(f"✅ 4 fichiers CSV à extraire")  # Modifié : 4 au lieu de 3
print(f"💾 Destination: {OUTPUT_BASE}")

✅ 13 tables PostgreSQL à extraire
✅ 4 fichiers CSV à extraire
💾 Destination: /home/jovyan/data/bronze


## 📊 PARTIE 1 : Extract PostgreSQL (tables originales)

In [4]:
def ingest_postgres_table(table_name):
    """Extrait une table PostgreSQL vers Bronze layer"""
    print(f"\n{'='*80}")
    print(f"🔄 Extract PostgreSQL: {table_name}")
    print(f"{'='*80}")
    
    start_time = time.time()
    
    try:
        df = spark.read.jdbc(
            url=JDBC_URL,
            table=f'"{table_name}"',
            properties=JDBC_PROPS
        )
        
        row_count = df.count()
        col_count = len(df.columns)
        
        print(f"📖 Lu: {row_count:,} lignes, {col_count} colonnes")
        
        # Ajout métadonnées
        df_with_meta = df \
            .withColumn("ingestion_timestamp", current_timestamp()) \
            .withColumn("ingestion_date", lit(datetime.now().strftime("%Y-%m-%d")))
        
        # Sauvegarde en Bronze
        output_path = f"{OUTPUT_BASE}/postgres/{table_name}"
        df_with_meta.write \
            .mode("overwrite") \
            .partitionBy("ingestion_date") \
            .parquet(output_path)
        
        elapsed = time.time() - start_time
        
        print(f"💾 Sauvegardé: {output_path}")
        print(f"⏱️  Temps: {elapsed:.2f}s")
        print(f"✅ {table_name} OK")
        
        return {
            "source": "PostgreSQL",
            "table": table_name,
            "rows": row_count,
            "cols": col_count,
            "time_sec": round(elapsed, 2),
            "status": "SUCCESS"
        }
        
    except Exception as e:
        print(f"❌ ERREUR: {str(e)}")
        return {
            "source": "PostgreSQL",
            "table": table_name,
            "rows": 0,
            "cols": 0,
            "time_sec": 0,
            "status": f"ERROR: {str(e)}"
        }

print("✅ Fonction d'ingestion PostgreSQL définie")

✅ Fonction d'ingestion PostgreSQL définie


In [5]:
# INGESTION POSTGRESQL
print("\n" + "="*80)
print("🚀 EXTRACTION POSTGRESQL - TABLES ORIGINALES")
print("="*80)

results = []

for table in POSTGRES_TABLES:
    result = ingest_postgres_table(table)
    results.append(result)

print("\n" + "="*80)
print("✅ EXTRACTION POSTGRESQL TERMINÉE")
print("="*80)


🚀 EXTRACTION POSTGRESQL - TABLES ORIGINALES

🔄 Extract PostgreSQL: Patient
📖 Lu: 100,000 lignes, 16 colonnes
💾 Sauvegardé: /home/jovyan/data/bronze/postgres/Patient
⏱️  Temps: 9.58s
✅ Patient OK

🔄 Extract PostgreSQL: Consultation
📖 Lu: 1,027,157 lignes, 9 colonnes
💾 Sauvegardé: /home/jovyan/data/bronze/postgres/Consultation
⏱️  Temps: 16.74s
✅ Consultation OK

🔄 Extract PostgreSQL: Diagnostic
📖 Lu: 15,490 lignes, 2 colonnes
💾 Sauvegardé: /home/jovyan/data/bronze/postgres/Diagnostic
⏱️  Temps: 1.17s
✅ Diagnostic OK

🔄 Extract PostgreSQL: Professionnel_de_sante
📖 Lu: 1,048,575 lignes, 8 colonnes
💾 Sauvegardé: /home/jovyan/data/bronze/postgres/Professionnel_de_sante
⏱️  Temps: 6.23s
✅ Professionnel_de_sante OK

🔄 Extract PostgreSQL: Mutuelle
📖 Lu: 254 lignes, 3 colonnes
💾 Sauvegardé: /home/jovyan/data/bronze/postgres/Mutuelle
⏱️  Temps: 1.32s
✅ Mutuelle OK

🔄 Extract PostgreSQL: Adher
📖 Lu: 96,671 lignes, 2 colonnes
💾 Sauvegardé: /home/jovyan/data/bronze/postgres/Adher
⏱️  Temps: 1.42s


## 📄 PARTIE 2 : Extract CSV BRUTS (directement depuis /DATA_2024/)

In [6]:
def ingest_csv_file(name, file_path, separator=";", encoding="UTF-8"):
    """Extrait un fichier CSV vers Bronze layer"""
    print(f"\n{'='*80}")
    print(f"🔄 Extract CSV: {name}")
    print(f"📁 Fichier: {file_path}")
    print(f"{'='*80}")
    
    start_time = time.time()
    
    try:
        # Lecture CSV BRUT
        df = spark.read \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .option("sep", separator) \
            .option("encoding", encoding) \
            .csv(file_path)
        
        row_count = df.count()
        col_count = len(df.columns)
        
        print(f"📖 Lu: {row_count:,} lignes, {col_count} colonnes")
        
        # Ajout métadonnées
        df_with_meta = df \
            .withColumn("ingestion_timestamp", current_timestamp()) \
            .withColumn("ingestion_date", lit(datetime.now().strftime("%Y-%m-%d")))
        
        # Sauvegarde en Bronze
        output_path = f"{OUTPUT_BASE}/csv/{name}"
        df_with_meta.write \
            .mode("overwrite") \
            .parquet(output_path)
        
        elapsed = time.time() - start_time
        
        print(f"💾 Sauvegardé: {output_path}")
        print(f"⏱️  Temps: {elapsed:.2f}s")
        print(f"✅ {name} OK")
        
        return {
            "source": "CSV",
            "table": name,
            "rows": row_count,
            "cols": col_count,
            "time_sec": round(elapsed, 2),
            "status": "SUCCESS"
        }
        
    except Exception as e:
        print(f"❌ ERREUR: {str(e)}")
        return {
            "source": "CSV",
            "table": name,
            "rows": 0,
            "cols": 0,
            "time_sec": 0,
            "status": f"ERROR: {str(e)}"
        }

print("✅ Fonction d'ingestion CSV définie")

✅ Fonction d'ingestion CSV définie


In [7]:
# 1. ÉTABLISSEMENTS DE SANTÉ
result = ingest_csv_file(
    name="etablissement_sante",
    file_path=f"{DATA_DIR}/Etablissement de SANTE/etablissement_sante.csv",
    separator=";"
)
results.append(result)


🔄 Extract CSV: etablissement_sante
📁 Fichier: /home/jovyan/DATA_2024/Etablissement de SANTE/etablissement_sante.csv
📖 Lu: 416,665 lignes, 24 colonnes
💾 Sauvegardé: /home/jovyan/data/bronze/csv/etablissement_sante
⏱️  Temps: 7.33s
✅ etablissement_sante OK


In [8]:
# 2. SATISFACTION 2019
result = ingest_csv_file(
    name="satisfaction_esatis48h_2019",
    file_path=f"{DATA_DIR}/Satisfaction/2019/resultats-esatis48h-mco-open-data-2019.csv",
    separator=";"
)
results.append(result)


🔄 Extract CSV: satisfaction_esatis48h_2019
📁 Fichier: /home/jovyan/DATA_2024/Satisfaction/2019/resultats-esatis48h-mco-open-data-2019.csv
📖 Lu: 1,152 lignes, 25 colonnes
💾 Sauvegardé: /home/jovyan/data/bronze/csv/satisfaction_esatis48h_2019
⏱️  Temps: 1.71s
✅ satisfaction_esatis48h_2019 OK


In [9]:
# 3. DÉCÈS 2019 UNIQUEMENT (FILTRÉ)
print(f"\n{'='*80}")
print(f"🔄 Extract CSV: deces (FILTRÉ 2019 UNIQUEMENT)")
print(f"📁 Fichier: {DATA_DIR}/DECES EN FRANCE/deces.csv")
print(f"{'='*80}")

start_time = time.time()

try:
    # Lecture CSV brut
    print("📖 Lecture du fichier CSV complet...")
    df_deces_raw = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .option("mode", "PERMISSIVE") \
        .option("multiLine", "false") \
        .csv(f"{DATA_DIR}/DECES EN FRANCE/deces.csv")
    
    # FILTRAGE : Ne garder que 2019
    # NOTE: La colonne s'appelle "date_deces" (pas "datdec")
    print("🔍 Filtrage des décès 2019 uniquement...")
    df_deces_full = df_deces_raw.filter(col("date_deces").startswith("2019"))
    
    # Repartitionner pour optimiser l'écriture
    df_deces_full = df_deces_full.repartition(10)
    
    row_count = df_deces_full.count()
    col_count = len(df_deces_full.columns)
    print(f"📊 Total 2019: {row_count:,} lignes, {col_count} colonnes")
    print(f"✅ FILTRÉ : Seulement données 2019 (réduction de 98%)")
    
    # Ajout métadonnées
    df_with_meta = df_deces_full \
        .withColumn("ingestion_timestamp", current_timestamp()) \
        .withColumn("ingestion_date", lit(datetime.now().strftime("%Y-%m-%d")))
    
    # Sauvegarde en Bronze (DONNÉES 2019 uniquement)
    output_path = f"{OUTPUT_BASE}/csv/deces_2019"
    df_with_meta.write \
        .mode("overwrite") \
        .option("compression", "snappy") \
        .parquet(output_path)
    
    elapsed = time.time() - start_time
    
    print(f"💾 Sauvegardé: {output_path}")
    print(f"⏱️  Temps: {elapsed:.2f}s")
    print(f"✅ deces 2019 OK ({row_count:,} lignes)")
    
    results.append({
        "source": "CSV",
        "table": "deces_2019",
        "rows": row_count,
        "cols": col_count,
        "time_sec": round(elapsed, 2),
        "status": "SUCCESS"
    })
    
except Exception as e:
    print(f"❌ ERREUR: {str(e)}")
    import traceback
    traceback.print_exc()
    results.append({
        "source": "CSV",
        "table": "deces_2019",
        "rows": 0,
        "cols": 0,
        "time_sec": 0,
        "status": f"ERROR: {str(e)}"
    })


🔄 Extract CSV: deces (FILTRÉ 2019 UNIQUEMENT)
📁 Fichier: /home/jovyan/DATA_2024/DECES EN FRANCE/deces.csv
📖 Lecture du fichier CSV complet...
🔍 Filtrage des décès 2019 uniquement...
📊 Total 2019: 620,626 lignes, 10 colonnes
✅ FILTRÉ : Seulement données 2019 (réduction de 98%)
💾 Sauvegardé: /home/jovyan/data/bronze/csv/deces_2019
⏱️  Temps: 37.80s
✅ deces 2019 OK (620,626 lignes)


In [10]:
# 4. DÉPARTEMENTS FRANÇAIS (référentiel géographique)
result = ingest_csv_file(
    name="departements",
    file_path=f"{DATA_DIR}/departements-francais.csv",
    separator=";"
)
results.append(result)


🔄 Extract CSV: departements
📁 Fichier: /home/jovyan/DATA_2024/departements-francais.csv
📖 Lu: 101 lignes, 4 colonnes
💾 Sauvegardé: /home/jovyan/data/bronze/csv/departements
⏱️  Temps: 1.22s
✅ departements OK


## 📊 RÉSUMÉ GLOBAL

In [11]:
# RÉSUMÉ
import pandas as pd

df_results = pd.DataFrame(results)
df_results

,source,table,rows,cols,time_sec,status
0,PostgreSQL,Patient,100000,16,9.58,SUCCESS
1,PostgreSQL,Consultation,1027157,9,16.74,SUCCESS
2,PostgreSQL,Diagnostic,15490,2,1.17,SUCCESS
3,PostgreSQL,Professionnel_de_sante,1048575,8,6.23,SUCCESS
4,PostgreSQL,Mutuelle,254,3,1.32,SUCCESS
5,PostgreSQL,Adher,96671,2,1.42,SUCCESS
6,PostgreSQL,Prescription,1003845,2,2.97,SUCCESS
7,PostgreSQL,Medicaments,15455,12,1.49,SUCCESS
8,PostgreSQL,Laboratoire,677,3,1.25,SUCCESS
9,PostgreSQL,Salle,201735,5,1.97,SUCCESS


In [12]:
# STATISTIQUES PAR SOURCE
success = df_results[df_results['status'] == 'SUCCESS']

print("\n📊 STATISTIQUES GLOBALES")
print("="*60)
print(f"✅ Tables extraites: {len(success)}/{len(results)}")
print(f"📊 Total lignes: {success['rows'].sum():,}")
print(f"⏱️  Temps total: {success['time_sec'].sum():.2f}s")

print("\n📦 Détail par source:")
for source in success['source'].unique():
    source_data = success[success['source'] == source]
    print(f"\n  {source}:")
    print(f"    - Tables: {len(source_data)}")
    print(f"    - Lignes: {source_data['rows'].sum():,}")
    print(f"    - Temps: {source_data['time_sec'].sum():.2f}s")

print("\n" + "="*60)
print("\n💾 Données sauvegardées dans: {OUTPUT_BASE}/")
print("  📂 bronze/postgres/ - 13 tables originales")
print("  📂 bronze/csv/ - 3 fichiers CSV")
print("="*60)


📊 STATISTIQUES GLOBALES
✅ Tables extraites: 17/17
📊 Total lignes: 4,712,928
⏱️  Temps total: 96.26s

📦 Détail par source:

  PostgreSQL:
    - Tables: 13
    - Lignes: 3,674,384
    - Temps: 48.20s

  CSV:
    - Tables: 4
    - Lignes: 1,038,544
    - Temps: 48.06s


💾 Données sauvegardées dans: {OUTPUT_BASE}/
  📂 bronze/postgres/ - 13 tables originales
  📂 bronze/csv/ - 3 fichiers CSV


In [13]:
---

## ✅ BRONZE LAYER - EXTRACTION COMPLÈTE

### 📦 Données extraites :

#### PostgreSQL (13 tables) :
- ✅ 100K patients
- ✅ 1M+ consultations
- ✅ 1M+ professionnels de santé
- ✅ **82K hospitalisations** (tables AAAA + date)
- ✅ Diagnostic, Prescription, Médicaments, etc.

#### CSV (4 fichiers lus DIRECTEMENT) :
- ✅ 416K établissements de santé
- ✅ 1K évaluations satisfaction 2019
- ✅ **600K décès 2019** (FILTRÉ depuis 25M - gain de 98%)
- ✅ 101 départements français

### Total : ~4 millions de lignes (optimisé)

### 🎯 Prochaine étape :

👉 **Notebook 02** : Transform Silver (Nettoyage, Anonymisation, Formats)

**Important** : Les décès sont maintenant **filtrés à 2019** pour performance et cohérence avec satisfaction 2019. Les hospitalisations proviennent des tables **AAAA + date**.

SyntaxError: invalid character '✅' (U+2705) (162563051.py, line 8)

---

## ✅ BRONZE LAYER - EXTRACTION COMPLÈTE

### 📦 Données extraites :

#### PostgreSQL (13 tables) :
- ✅ 100K patients
- ✅ 1M+ consultations
- ✅ 1M+ professionnels de santé
- ✅ Diagnostic, Prescription, Médicaments, etc.

#### CSV (3 fichiers lus DIRECTEMENT) :
- ✅ 416K établissements de santé
- ✅ 1K évaluations satisfaction 2019
- ✅ **25M décès COMPLET** (toutes années - SANS FILTRAGE)

### Total : ~29 millions de lignes

### 🎯 Prochaine étape :

👉 **Notebook 02** : Transform Silver (Nettoyage, Anonymisation, Formats)

**Important** : Les CSV sont maintenant en Bronze sous forme **BRUTE INTÉGRALE**. Le filtrage temporel (si nécessaire) sera appliqué dans Silver ou Gold selon les besoins métier.

In [14]:
# Dans une cellule Jupyter
df = spark.read.parquet("/home/jovyan/data/bronze/postgres/Patient")
print(f"Patients en Bronze : {df.count():,}")
df.show(5)

Patients en Bronze : 100,000
+----------+----------+--------+------+--------------------+-----------------+-----------+----+--------------------+--------------+----------+---+----------------+--------------+-----+------+--------------------+--------------+
|Id_patient|       Nom|  Prenom|  Sexe|             Adresse|            Ville|Code_postal|Pays|               EMail|           Tel|      Date|Age|        Num_Secu|Groupe_sanguin| Poid|Taille| ingestion_timestamp|ingestion_date|
+----------+----------+--------+------+--------------------+-----------------+-----------+----+--------------------+--------------+----------+---+----------------+--------------+-----+------+--------------------+--------------+
|         1|Christabel|  Tougas|female|12 rue du Faubour...|       THIONVILLE|      57100|  FR|ChristabelTougas@...|03.85.46.00.55|  4/6/1980| 41|5571905089387417|            O+| 54.3|   162|2025-10-23 18:39:...|    2025-10-23|
|         2|  Lorraine|   Lebel|female|   21 rue Jean Vilar